In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import scipy.stats as stats
from lifelines.plotting import add_at_risk_counts

In [31]:
# input files/dir
ANNOTATED_MUTATIONS = "../data/annotated_snv_mv_indels_by_cancer_subtype"
GENOME_WIDE_MUTATIONS = "../data/genome_wide_mutation_data.tsv"
DRIVER_GENES = "../data/driver_genes"
CLINICAL_DATA = "../data/datasets/PCAWG/clinical_data/pcawg_donor_clinical_data.csv"
WHITELISTED_SAMPLES = "../data/datasets/PCAWG/supplementary Tables/Supplementary Table 1.csv"
SURVIVAL_DATA = "../data/datasets/TCGA/TCGA_survival_outcome.csv"
CANCER_GENE_TYPES = "../metadata/cancer_genes.tsv"

# output files/ directories
OS_ANALYSIS = "../results/PCAWG/OS_pancancer_gene"
PLOT_DATA_DIR = "../plot_data/OS_gene"
os.makedirs(OS_ANALYSIS, exist_ok=True)
os.makedirs(PLOT_DATA_DIR, exist_ok=True)
    
CANCER_TYPES = os.listdir(DRIVER_GENES) # cancer types with driver genes
CANCER_TYPES = [cancer_type for cancer_type in CANCER_TYPES if cancer_type.endswith(".tsv")]
CANCER_TYPES = [cancer_type.replace(".tsv", "") for cancer_type in CANCER_TYPES]
CANCER_TYPES.remove("Pancancer")
print(len(CANCER_TYPES), CANCER_TYPES)

31 ['CNS-Oligo', 'Kidney-ChRCC', 'Prost-AdenoCA', 'Kidney-RCC', 'Stomach-AdenoCA', 'CNS-Medullo', 'Thy-AdenoCA', 'Myeloid-MPN', 'Bone-Leiomyo', 'Lymph-BNHL', 'Myeloid-AML', 'Lung-AdenoCA', 'CNS-GBM', 'Head-SCC', 'Breast-AdenoCa', 'Ovary-AdenoCA', 'CNS-PiloAstro', 'Cervix-SCC', 'Liver-HCC', 'Bone-Osteosarc', 'Biliary-AdenoCA', 'Skin-Melanoma', 'Lung-SCC', 'Lymph-CLL', 'Panc-Endocrine', 'Bladder-TCC', 'Panc-AdenoCA', 'ColoRect-AdenoCA', 'Breast-LobularCa', 'Eso-AdenoCa', 'Uterus-AdenoCA']


In [32]:
whitelisted_data = pd.read_csv(WHITELISTED_SAMPLES, sep=",", header=0)
whitelisted_samples = whitelisted_data["tumour_specimen_aliquot_id"].unique().tolist()
print(f"Number of unique samples in the whitelist: {len(whitelisted_samples)}")

Number of unique samples in the whitelist: 2583


In [33]:
# get all mutations in driver genes
tumor_mut_df = pd.DataFrame()
for cancer_type in CANCER_TYPES:
	tumor_mut_cancer_df = pd.read_csv(f"{ANNOTATED_MUTATIONS}/{cancer_type}.tsv", sep="\t")
	driver_genes = pd.read_csv(os.path.join(DRIVER_GENES, cancer_type + ".tsv"), sep="\t")
	driver_genes_list = driver_genes["gene"].tolist()
	tumor_mut_cancer_df = tumor_mut_cancer_df[tumor_mut_cancer_df["gene"].isin(driver_genes_list)]
	tumor_mut_cancer_df = pd.merge(tumor_mut_cancer_df, driver_genes[["gene", "gene_length"]], on="gene", how="left")

	# get genome-wide mutation data
	genome_wide_mutations = pd.read_csv(GENOME_WIDE_MUTATIONS, sep="\t")
	tumor_mut_cancer_df = pd.merge(tumor_mut_cancer_df, genome_wide_mutations, on=["Tumor_Sample_Barcode"], how="left")
	tumor_mut_cancer_df["cancer_type"] = cancer_type
	tumor_mut_df = pd.concat([tumor_mut_df, tumor_mut_cancer_df], ignore_index=True)

print(tumor_mut_df.shape)
# filter out samples that are not in the whitelist
tumor_mut_df = tumor_mut_df[tumor_mut_df["Tumor_Sample_Barcode"].isin(whitelisted_samples)]
print(tumor_mut_df.shape)

/var/folders/kb/00sgh9j520x459vtmckmbyjw0000gp/T/ipykernel_98115/260916671.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tumor_mut_cancer_df = pd.read_csv(f"{ANNOTATED_MUTATIONS}/{cancer_type}.tsv", sep="\t")
/var/folders/kb/00sgh9j520x459vtmckmbyjw0000gp/T/ipykernel_98115/260916671.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tumor_mut_cancer_df = pd.read_csv(f"{ANNOTATED_MUTATIONS}/{cancer_type}.tsv", sep="\t")
/var/folders/kb/00sgh9j520x459vtmckmbyjw0000gp/T/ipykernel_98115/260916671.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tumor_mut_cancer_df = pd.read_csv(f"{ANNOTATED_MUTATIONS}/{cancer_type}.tsv", sep="\t")
/var/folders/kb/00sgh9j520x459vtmckmbyjw0000gp/T/ipykernel_98115/260916671.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tumo

(90258, 17)
(82512, 17)


In [34]:
# get clinical data
clinical_data_df = pd.read_csv(CLINICAL_DATA)
clinical_data_df = clinical_data_df[["icgc_donor_id", "submitted_donor_id", "donor_sex"]]
clinical_data_df = clinical_data_df.rename(columns={
    "icgc_donor_id": "Patient_ID", 
    "submitted_donor_id": "bcr_patient_barcode"
})
clinical_data_df.dropna(inplace=True)
clinical_data_df.drop_duplicates(inplace=True)
print(clinical_data_df.shape)

(2809, 3)


In [35]:
# get survival outcomes data
survival_data_df = pd.read_csv(SURVIVAL_DATA)
survival_data_df.reset_index(inplace=True)
survival_data_df.drop(columns=["index", "Unnamed: 0"], inplace=True)
print(survival_data_df.shape)

(11160, 33)


In [36]:
# merge tumor data with clinical data
tumor_data_merged_df = pd.merge(tumor_mut_df, clinical_data_df, on="Patient_ID", how="left")
print(tumor_data_merged_df.shape)

# merge tumor data with survival data
tumor_survival_data_merged_df = pd.merge(tumor_data_merged_df, survival_data_df, on="bcr_patient_barcode", how="left")
print(tumor_survival_data_merged_df.shape)

(82512, 19)
(82512, 51)


In [37]:
def label_cadd_score(cadd_score):
    if pd.isnull(cadd_score):
        return "no cadd score"
    elif cadd_score <= 0:
        return "low cadd score"
    else:
        return "high cadd score"

# filter out indels and MNVs without CADD scores
print(tumor_survival_data_merged_df.shape)
tumor_survival_data_merged_df_witout_cadd = tumor_survival_data_merged_df[tumor_survival_data_merged_df["CADD_score_raw"].isnull()]
tumor_survival_data_merged_df = tumor_survival_data_merged_df[tumor_survival_data_merged_df["CADD_score_raw"].notnull()]
print(tumor_survival_data_merged_df.shape)

# normalize CADD scores
tumor_survival_data_merged_df["CADD_score_normalized"] = tumor_survival_data_merged_df.groupby("gene")["CADD_score_raw"].transform(lambda x: (x - x.mean()) / x.std())
tumor_survival_data_merged_df = pd.concat([tumor_survival_data_merged_df, tumor_survival_data_merged_df_witout_cadd], ignore_index=True)

# label mutations by cadd score
tumor_survival_data_merged_df["CADD_score_label"] = tumor_survival_data_merged_df["CADD_score_normalized"].apply(label_cadd_score)

(82512, 51)
(77042, 51)


## Overall Survival (OS) analysis

In [38]:
# tumor samples for which we have survival data
tumor_survival_data_merged_df.dropna(subset=["OS", "OS.time"], inplace=True)
print(tumor_survival_data_merged_df.shape)

# mutation density by driver status
tumor_OS_df = tumor_survival_data_merged_df[["Tumor_Sample_Barcode", "gene", "mutation", "driver", "has_driver", "OS", "OS.time", "CADD_score_label"]]
tumor_OS_df["has_driver"] = tumor_OS_df["has_driver"].apply(lambda x: "Passengers in presence of driver" if x else "Passengers in absence of driver")
tumor_OS_df["has_driver"] = tumor_OS_df.apply(lambda x: "Drivers" if x["driver"] == True else x["has_driver"], axis=1)
tumor_OS_mut_density = tumor_OS_df.groupby(["Tumor_Sample_Barcode", "gene", "has_driver"]).agg({
    "mutation": "count",
    "OS": "first",
    "CADD_score_label": lambda x: list(set(x)),
    "OS.time": "first",
}).reset_index()
tumor_OS_mut_density["CADD_score_label"] = tumor_OS_mut_density["CADD_score_label"].apply(lambda x: "high cadd score" if "high cadd score" in x else "low cadd score" if "low cadd score" in x else "no cadd score")
tumor_OS_mut_density.rename(columns={"mutation": "mutation_count"}, inplace=True)
print(tumor_OS_mut_density.shape)
tumor_OS_mut_density = tumor_OS_mut_density[tumor_OS_mut_density["has_driver"] != "Passengers in presence of driver"]
print(tumor_OS_mut_density.shape)

(36659, 53)
(3610, 7)
(3320, 7)


/var/folders/kb/00sgh9j520x459vtmckmbyjw0000gp/T/ipykernel_98115/2508359109.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tumor_OS_df["has_driver"] = tumor_OS_df["has_driver"].apply(lambda x: "Passengers in presence of driver" if x else "Passengers in absence of driver")
/var/folders/kb/00sgh9j520x459vtmckmbyjw0000gp/T/ipykernel_98115/2508359109.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tumor_OS_df["has_driver"] = tumor_OS_df.apply(lambda x: "Drivers" if x["driver"] == True else x["has_drive

In [39]:
# list of all tumor samples for which we have survival data
tumor_list = tumor_OS_mut_density["Tumor_Sample_Barcode"].unique().tolist()
print(f"Number of tumor samples for which we have survival data: {len(tumor_list)}")

# list of all genes
gene_list = tumor_OS_mut_density["gene"].unique().tolist()
print(f"Number of genes: {len(gene_list)}")

# create a map of survival data for each tumor
tumor_survival_map = {}
for index, row in tumor_OS_mut_density.iterrows():
    tumor_survival_map[row["Tumor_Sample_Barcode"]] = {
        "OS": row["OS"],
        "OS.time": row["OS.time"]
    }
print(len(tumor_survival_map), tumor_survival_map)

# create a map of mutation counts for each tumor-gene-has_driver combination
tumor_gene_has_driver_map = {}
for index, row in tumor_OS_mut_density.iterrows():
    tumor_gene_has_driver_map[(row["Tumor_Sample_Barcode"], row["gene"], row["has_driver"], row["CADD_score_label"])] = {
        "mutation_count": row["mutation_count"]
    }
print(len(tumor_gene_has_driver_map), tumor_gene_has_driver_map)

Number of tumor samples for which we have survival data: 710
Number of genes: 108
710 {'00493087-9d9d-40ca-86d5-936f1b951c93': {'OS': 0.0, 'OS.time': 1201.0}, '00aa769d-622c-433e-8a8a-63fb5c41ea42': {'OS': 0.0, 'OS.time': 255.0}, '00bf0350-8c7c-4b9e-8143-13ea2dc1122f': {'OS': 0.0, 'OS.time': 617.0}, '00db4dc2-3ec7-4ff9-9233-d69c8c8a607f': {'OS': 0.0, 'OS.time': 776.0}, '01658141-8398-4585-9f0f-8355dd9b0604': {'OS': 0.0, 'OS.time': 1925.0}, '0176cf1d-0760-4769-a493-277f4bb7585e': {'OS': 1.0, 'OS.time': 303.0}, '0192d529-7340-45d8-a5f0-249cbb11ca19': {'OS': 0.0, 'OS.time': 531.0}, '020fab36-c7de-4933-b2bf-dc7b019a1326': {'OS': 0.0, 'OS.time': 842.0}, '02c6a893-49c5-49d1-8eb1-195021e70d52': {'OS': 0.0, 'OS.time': 1521.0}, '0332b017-17d5-4083-8fc4-9d6f8fdbbbde': {'OS': 0.0, 'OS.time': 470.0}, '0385961e-ea99-40b2-ad79-6872bc30d8a1': {'OS': 0.0, 'OS.time': 2554.0}, '03c88506-d72e-4a44-a34e-a7f0564f1799': {'OS': 1.0, 'OS.time': 802.0}, '03ced0ce-186a-4349-8d98-572c2bc90382': {'OS': 1.0, 'OS.t

In [40]:
tumor_gene_driver_data = []
for tumor in tumor_list:
    cancer_type = tumor_survival_data_merged_df[tumor_survival_data_merged_df["Tumor_Sample_Barcode"] == tumor]["cancer_type"].values[0]
    for gene in gene_list:
        if (tumor, gene, "Drivers", "low cadd score") in tumor_gene_has_driver_map or (tumor, gene, "Drivers", "high cadd score") in tumor_gene_has_driver_map:
            tumor_gene_driver_data.append({
                "Tumor_Sample_Barcode": tumor,
                "gene": gene,
                "mutation_status": "Has drivers",
                "OS": tumor_survival_map[tumor]["OS"],
                "OS.time": tumor_survival_map[tumor]["OS.time"],
                "cancer_type": cancer_type
            })
        elif (tumor, gene, "Passengers in absence of driver", "low cadd score") in tumor_gene_has_driver_map:
            tumor_gene_driver_data.append({
                "Tumor_Sample_Barcode": tumor,
                "gene": gene,
                "mutation_status": "Has passengers with low cadd scores but no drivers",
                "OS": tumor_survival_map[tumor]["OS"],
                "OS.time": tumor_survival_map[tumor]["OS.time"],
                "cancer_type": cancer_type
            })
        elif (tumor, gene, "Passengers in absence of driver", "high cadd score") in tumor_gene_has_driver_map:
            tumor_gene_driver_data.append({
                "Tumor_Sample_Barcode": tumor,
                "gene": gene,
                "mutation_status": "Has passengers with high cadd scores but no drivers",
                "OS": tumor_survival_map[tumor]["OS"],
                "OS.time": tumor_survival_map[tumor]["OS.time"],
                "cancer_type": cancer_type
            })
        elif (tumor, gene, "Passengers in absence of driver", "no cadd score") in tumor_gene_has_driver_map:
            tumor_gene_driver_data.append({
				"Tumor_Sample_Barcode": tumor,
				"gene": gene,
				"mutation_status": "Has passengers but no drivers",
				"OS": tumor_survival_map[tumor]["OS"],
				"OS.time": tumor_survival_map[tumor]["OS.time"],
				"cancer_type": cancer_type
			})
        else:
            tumor_gene_driver_data.append({
                "Tumor_Sample_Barcode": tumor,
                "gene": gene,
                "mutation_status": "Has no mutations",
                "OS": tumor_survival_map[tumor]["OS"],
                "OS.time": tumor_survival_map[tumor]["OS.time"],
                "cancer_type": cancer_type
            })
tumor_gene_driver_data = pd.DataFrame(tumor_gene_driver_data)
print(tumor_gene_driver_data.shape)

(76680, 6)


## Gene-level analysis

In [41]:
gene_data = tumor_gene_driver_data.groupby(["gene", "mutation_status"])["Tumor_Sample_Barcode"].nunique().reset_index()
gene_data.rename(columns={"Tumor_Sample_Barcode": "sample_count"}, inplace=True)

# filter for genes with minimum samples in each category
MIN_SAMPLE_COUNT = 10
MIN_DRIVER_COUNT = 5
genes_without_mutations = gene_data[gene_data["mutation_status"] == "Has no mutations"].groupby("gene")["sample_count"].sum().reset_index()
genes_without_mutations = genes_without_mutations[genes_without_mutations["sample_count"] >= MIN_SAMPLE_COUNT]["gene"].tolist()

categories = ["Has passengers with low cadd scores but no drivers", "Has passengers with high cadd scores but no drivers", "Has passengers but no drivers"]
genes_with_only_passengers = gene_data[gene_data["mutation_status"].isin(categories)].groupby("gene")["sample_count"].sum().reset_index()
genes_with_only_passengers = genes_with_only_passengers[genes_with_only_passengers["sample_count"] >= MIN_SAMPLE_COUNT]["gene"].tolist()

genes_with_drivers = gene_data[gene_data["mutation_status"] == "Has drivers"].groupby("gene")["sample_count"].sum().reset_index()
genes_with_drivers = genes_with_drivers[genes_with_drivers["sample_count"] >= MIN_DRIVER_COUNT]["gene"].tolist()

genes_of_interest = set(genes_without_mutations) & set(genes_with_only_passengers) & set(genes_with_drivers)
print(len(genes_of_interest))

tumor_gene_has_driver_data = tumor_gene_driver_data[tumor_gene_driver_data["gene"].isin(genes_of_interest)]
tumor_analysis_list = tumor_gene_has_driver_data["Tumor_Sample_Barcode"].unique().tolist()
print(f"Number of tumor samples shortlisted for survival analysis: {len(tumor_analysis_list)}")

22
Number of tumor samples shortlisted for survival analysis: 710


In [42]:
# custom color palette
colors = ["#CC6677", "#106C9A", "#529267", "#eebb44"]
sns.palettes.color_palette(colors)
cmap = sns.color_palette(colors)

# plot Kaplan-Meier survival curves
for gene in genes_of_interest:
	tumor_gene_has_driver_data_selected = tumor_gene_has_driver_data[tumor_gene_has_driver_data["gene"] == gene]

	# KAPLAN-MEIER PLOTS
	plt.figure(figsize=(8, 10))
	sns.set_style("ticks")

	# samples with no mutations in the gene
	kmf_control = KaplanMeierFitter()
	group0 = tumor_gene_has_driver_data_selected[tumor_gene_has_driver_data_selected["mutation_status"] == "Has no mutations"]
	kmf_control_result = kmf_control.fit(group0["OS.time"], group0["OS"], label='No mutations')
	ax = kmf_control_result.plot_survival_function(color=cmap[0], ci_show=False, legend=False)
	
	# "no mutations" vs "passengers but no drivers"
	categories = ["Has passengers with low cadd scores but no drivers", "Has passengers with high cadd scores but no drivers"]
	group1 = tumor_gene_has_driver_data_selected[tumor_gene_has_driver_data_selected["mutation_status"].isin(categories)]
	if len(group1) > 0:
		kmf_exp1 = KaplanMeierFitter()
		kmf_exp1_result = kmf_exp1.fit(group1["OS.time"], group1["OS"], label='Passenger mutations only')
		kmf_exp1_result.plot_survival_function(color=cmap[1], ci_show=False, legend=False)
		results1 = logrank_test(group0['OS.time'], group1['OS.time'], event_observed_A=group0['OS'], event_observed_B=group1['OS'])
		p_value1 = min(results1.p_value * len(genes_of_interest), 1)  # Bonferroni correction for multiple testing
		color1 = 'red' if p_value1 < 0.05 else 'black'
		plt.text(0.72, 0.61, rf"$\it{{p}}_1$={p_value1:.3f}", 
			transform=plt.gca().transAxes, fontsize=17, color=color1)

	# "no mutations" vs "drivers"
	group2 = tumor_gene_has_driver_data_selected[tumor_gene_has_driver_data_selected["mutation_status"] == "Has drivers"]
	if len(group2) > 0:
		kmf_exp2 = KaplanMeierFitter()
		kmf_exp2_result = kmf_exp2.fit(group2["OS.time"], group2["OS"], label='Drivers')
		kmf_exp2_result.plot_survival_function(color=cmap[2], ci_show=False, legend=False)
		results2 = logrank_test(group0['OS.time'], group2['OS.time'], event_observed_A=group0['OS'], event_observed_B=group2['OS'])
		p_value2 = min(results2.p_value * len(genes_of_interest), 1)  # Bonferroni correction for multiple testing
		color3 = 'red' if p_value2 < 0.05 else 'black'
		plt.text(0.72, 0.52, rf"$\it{{p}}_2$={p_value2:.3f}", 
				transform=plt.gca().transAxes, fontsize=17, color=color3)

	fitters = [kmf_control]
	if 'kmf_exp1' in locals():
		fitters.append(kmf_exp1)
	if 'kmf_exp2' in locals():
		fitters.append(kmf_exp2)
	add_at_risk_counts(*fitters, ax=ax, fontsize=16)

	ax.set_ylabel('Probability', labelpad=10, fontsize=18)
	ax.set_xlabel('Time (days)', labelpad=10, fontsize=18)
	ax.tick_params(axis='x', labelsize=18)
	ax.tick_params(axis='y', labelsize=18)
	ax.legend(fontsize=16, loc='upper right')
	plt.title(gene, fontsize=20, fontstyle='italic')
	plt.tight_layout()
	plt.savefig(f"{OS_ANALYSIS}/{gene}.png", dpi=300)
	print(gene, group0.shape, group1.shape)
	plt.close()

	# source data
	tumor_gene_has_driver_data_selected[["OS", "OS.time", "mutation_status"]].to_csv(f"{PLOT_DATA_DIR}/{gene}_OS.tsv", sep="\t", index=False)

NFE2L2 (649, 6) (39, 6)
TP53 (420, 6) (18, 6)
PIK3CA (562, 6) (53, 6)
ARHGAP35 (687, 6) (17, 6)
PBRM1 (686, 6) (14, 6)
ARID1A (669, 6) (30, 6)
KMT2D (671, 6) (26, 6)
APC (660, 6) (22, 6)
ATM (674, 6) (25, 6)
EGFR (660, 6) (30, 6)
NOTCH1 (672, 6) (19, 6)
BRAF (671, 6) (14, 6)
SMAD4 (684, 6) (14, 6)
PTEN (671, 6) (23, 6)
FBXW7 (660, 6) (29, 6)
FAT1 (640, 6) (60, 6)
TERT (604, 6) (24, 6)
RB1 (662, 6) (38, 6)
CTNNB1 (675, 6) (15, 6)
KDM6A (695, 6) (9, 6)
CREBBP (677, 6) (21, 6)
CDKN2A (679, 6) (11, 6)


## Testing significant genes for CNA effects

In [43]:
genes = ["EGFR", "TERT", "RB1"]
for gene in genes:
    selected_gene_df = tumor_gene_has_driver_data[tumor_gene_has_driver_data["gene"] == gene]
    selected_gene_df = selected_gene_df.merge(genome_wide_mutations, on=["Tumor_Sample_Barcode", "cancer_type"], how="left")
    selected_gene_df["mutation_status_group"] = selected_gene_df["mutation_status"].apply(lambda x: "Passengers with no drivers" if "passengers" in x else x)

    # difference in cna burden between samples with no mutations and samples with passengers but no drivers
    group_no_mutations = selected_gene_df[selected_gene_df["mutation_status"] == "Has no mutations"]["cna_burden"].dropna()
    group_passengers_no_drivers = selected_gene_df[selected_gene_df["mutation_status"].str.contains("passengers")]["cna_burden"].dropna()
    stat, p_value = stats.mannwhitneyu(group_no_mutations, group_passengers_no_drivers, alternative='two-sided')
    print(f"Mann-Whitney U test results for {gene}: statistic = {stat}, p-value = {p_value}")


Mann-Whitney U test results for EGFR: statistic = 8933.0, p-value = 0.14081651580610513
Mann-Whitney U test results for TERT: statistic = 6549.0, p-value = 0.4229446602759639
Mann-Whitney U test results for RB1: statistic = 13881.0, p-value = 0.8062475406613588
